# Particle Set


Simulate trajectories of a particle cloud in a two-dimensional flow field.
A doubly-periodic domain and randomly-generated flow fields are initially used.
For additional documentation e.g. see :
[1](https://JuliaClimate.github.io/IndividualDisplacements.jl/dev/),
[2](https://JuliaClimate.github.io/MeshArrays.jl/dev/),
[3](https://docs.juliadiffeq.org/latest/solvers/ode_solve.html),
[4](https://en.wikipedia.org/wiki/Displacement_(vector))

Exercises:
- change the initial distribution of partices
- increase the duration of the trajectories simulation
- treat the non-periodic domain case by padding `u,v` with zeros
- replace `u,v` with your own two-dimensional flow fields

![particles in random flow](https://github.com/JuliaClimate/IndividualDisplacements.jl/raw/master/examples/figs/RandomFlow.gif)

## 1. Import Software

In [1]:
using IndividualDisplacements, DataFrames
p=dirname(pathof(IndividualDisplacements))
include(joinpath(p,"../examples/helper_functions.jl"))

Main.##256.isosurface

## 2. Flow Field

The `u,v` arrays below can be replaced with any other pair provided by the user.
A couple of important considerations, however:

- `u,v` are staggered on a C-grid; by `-0.5` grid point in direction `1` for `u` (`2` for `v`)
 from the grid cell center (0.5,0.5)
- `u,v` here derive from streamfunction `ϕ`, defined at the corner point, which ensures that
 the resulting `u,v` is non-divergent, purely rotational, over the C-grid domain.
In brief:

```
u=-(circshift(ϕ, (0,-1))-ϕ)
v=(circshift(ϕ, (-1,0))-ϕ)
```

In [2]:
u,v,ϕ=setup_random_flow()

([-0.003136114850947759 -0.00039890201873851033 … -0.01003877757985113 -0.0053716628567951775; -0.00418228011237875 -0.0013669132909075034 … -0.009504183712731942 -0.007352429338325847; … ; 7.330834686411491e-5 0.004495030651043945 … -0.009801279208166477 -0.00218990242488859; -0.00042493545758555754 0.0008567513623177862 … -0.00726004951747595 -0.005427281346188521], [-0.008364306580490744 -0.007318141319059753 … -0.009810479194902227 -0.010345073062021414; 0.002880109640220549 0.002692365849407164 … 0.00021663531669190161 0.0019856117656421625; … ; -0.020382942129641222 -0.01988469832519155 … -0.02107909136025063 -0.023620321050941154; -0.017922780920488685 -0.015211601527126483 … -0.02064589049347052 -0.01786716243109534], [0.0012863836663445025 0.0044224985172922615 … -0.014124056770301804 -0.004085279190450675; -0.007077922914146242 -0.0028956428017674917 … -0.02393453596520403 -0.014430352252472089; … ; 0.03959210671647441 0.039518798369610295 … 0.027600925083419343 0.03740220429

If user were to start with collocated velocity (`uC,vC` at the grid cell center) then
one can easily obtain the staggered velocity (`u,v`) as follows. These may contain both
[rotational and divergent](https://en.wikipedia.org/wiki/Helmholtz_decomposition) components.

```
u=0.5*(circshift(uC, (0,1))+uC)
v=0.5*(circshift(vC, (1,0))+vC)
```

## 3. Initialize Individuals

For example, we can initialize 100 particles within a central subdomain as follows.

In [3]:
np,nq=size(u)
x=np*(0. .+ 1.0*rand(1000))
y=nq*(0. .+ 1.0*rand(1000));

The `setup_point_cloud` function then wraps everything in the `Individuals` data structure.

In [4]:
𝐼=setup_point_cloud(u,v,X=x,Y=y)
𝐼.𝑃.𝑇[2]=10.
𝐼.🔴

,ID,x,y,t
,Int64,Float64,Float64,Float64


## 3. Compute Trajectories

The time period is `𝐼.𝑃.𝑇` by default, unless `∫!(𝐼,𝑇)` is called instead.

In [5]:
∫!(𝐼)

1×1000 Array{Array{Float64,1},2}:
 [8.68419, 2.31735, 1.0]  [3.43505, 7.65948, 1.0]  …  [6.61582, 9.04836, 1.0]

## 4. Plot Results

For example, generate a simple animation:

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*